In [96]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Load DNS events

In [84]:
data = pd.read_json('data/events.json', lines=True)

dns_events = data[data['event_type'] == 'dns']

print(f'{len(dns_events)} DNS events found')
dns_events.sample(2)

15749 DNS events found


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
119304,2017-07-22T18:29:17.358830-0500,2.019680e+15,504555.0,dns,150.0,192.168.207.4,53.0,192.168.205.188,60984.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140955,2017-07-22T18:33:28.118093-0500,6.674917e+14,565680.0,dns,150.0,192.168.207.4,53.0,192.168.205.188,50816.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Normalization

In [85]:
dns_df = pd.json_normalize(dns_events.to_dict(orient="records"), sep="_")

print(f'Dataset Shape: {dns_df.shape}')

Dataset Shape: (15749, 35)


### Filtering

In [93]:
dns_df = dns_df[dns_df['dns_rrtype'] == 'A']
unique_domains_df = dns_df.drop_duplicates(subset=['dns_rrname'])

print(f'Found {len(dns_df)} A records')
print(f'Found {dns_df["dns_rrname"].nunique()} unique domains')

Found 2849 A records
Found 177 unique domains


### TLDs

In [88]:
def get_tld(domain):
    parts = domain.split('.')
    
    if len(parts) <= 2:
        return domain
    
    common_tlds = ['com', 'org', 'net', 'edu', 'gov', 'mil', 'io', 'ai', 'co']
    
    if parts[-1] in common_tlds:
        return '.'.join(parts[-2:])
    
    country_tlds = ['uk', 'au', 'ca', 'de', 'jp', 'fr', 'es', 'it', 'ru', 'cn', 'br', 'mx']
    
    if parts[-1] in country_tlds and len(parts) >= 3:
        if parts[-2] in ['co', 'com', 'org', 'net', 'ac', 'gov']:
            return '.'.join(parts[-3:])
    
    if len(parts) > 2 and parts[-1] not in common_tlds + country_tlds:
        return parts[-1]
    
    return '.'.join(parts[-2:])

In [105]:
unique_domains_df['domain_tld'] = unique_domains_df['dns_rrname'].apply(get_tld)

domain_tld_df = unique_domains_df[['domain_tld']]

print(f"Unique TLDs: {domain_tld_df['domain_tld'].nunique()}")

domain_tld_df.to_csv('data/tld.csv', index=False)


Unique TLDs: 104
